# Imports & Installations

In [ ]:
!nvidia-smi

Tue Feb  4 11:29:35 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   30C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [ ]:
!pip install git+https://github.com/huggingface/transformers.git

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-pyr00xfv
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-pyr00xfv
  Resolved https://github.com/huggingface/transformers.git to commit bc9a6d8302334ae08d505437ab3f361af777956c
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.49.0.dev0-py3-none-any.whl size=10633398 sha256=d48f4e48e80168a61c740d033ea73b8345fb93e8783a83b18c41dfc0d1e5fed8
  Stored in directory: /tmp/pip-ephem-wheel-cache-ggjyzw8_/wheels/32/4b/78/f195c684dd3a9ed21f3b39fe8f85b48df7918581b6437be143
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.47.1
    Uninstalling transformers-4.47.1:
      Successfully uninstalled transformers-4.47.1


In [ ]:
!pip install datasets wandb accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 108.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 82.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 38.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
class Config:
    def __init__(self):

        # Base model (ModernBERT)
        self.model_name = "/content/drive/MyDrive/ModernEgyBert/egyptian_modernbert_fineweb2/checkpoint-60000"
        #"answerdotai/ModernBERT-base"

        # Tokenization
        self.max_length = 512
        self.new_tokenizer_vocab_size= 64000
        # MLM
        self.mlm_probability = 0.15

        # Training Arguments
        self.base_dir = "/content/drive/MyDrive/ModernEgyBert"
        self.output_dir = self.base_dir + "/egyptian_modernbert_fineweb2"
        self.num_train_epochs = 3
        self.per_device_train_batch_size = 32
        self.per_device_eval_batch_size = 8
        self.evaluation_strategy = "steps"
        self.eval_steps = 5000
        self.logging_steps = 100
        self.save_steps = 5000
        self.save_total_limit = 2
        self.learning_rate = 5e-5
        self.warmup_steps = 500
        self.weight_decay = 0.01
        self.report_to = "wandb"
        self.run_name = "modernbert-egyptian-handle-english"
        self.overwrite_output_dir = True


In [ ]:
config = Config()

In [ ]:
import wandb
import re
import torch
from datasets import DatasetDict, Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForMaskedLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)

In [ ]:
wandb.login()

In [ ]:
from datasets import load_dataset
dataset = load_dataset("HuggingFaceFW/fineweb-2","arz_Arab")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/844k [00:00<?, ?B/s]

000_00000.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

000_00000.parquet:   0%|          | 0.00/1.51G [00:00<?, ?B/s]

Generating test split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def remove_english_words(text):
    text_no_eng = re.sub(r"\b[a-zA-Z]+\b", "", text)
    return text_no_eng.strip()

def preprocess_examples(examples):
    new_texts = []
    for t in examples["text"]:
        new_texts.append(remove_english_words(t))
    # Return a dictionary with the same column name
    return {"text": new_texts}

# Apply this preprocessing to train & test
dataset = dataset.map(preprocess_examples, batched=True)


Map:   0%|          | 0/9216 [00:00<?, ? examples/s]

Map:   0%|          | 0/1410134 [00:00<?, ? examples/s]

In [ ]:
base_tokenizer = AutoTokenizer.from_pretrained(config.model_name, use_fast=True)

# We'll create a batch iterator to feed lines to the tokenizer trainer
def batch_iterator(ds, batch_size=1000):
    for i in range(0, len(ds), batch_size):
        yield ds[i : i + batch_size]["text"]

# Train a new tokenizer from the cleaned training text
train_iterator = batch_iterator(dataset["train"])
new_tokenizer = base_tokenizer.train_new_from_iterator(
    text_iterator=train_iterator,
    vocab_size=config.new_tokenizer_vocab_size,
    show_progress=True
)

# (Optionally) Save the tokenizer now, if you like
new_tokenizer.save_pretrained(f"{config.base_dir}/tokenizer")

tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.13M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

('/content/drive/MyDrive/ModernEgyBert/tokenizer/tokenizer_config.json',
 '/content/drive/MyDrive/ModernEgyBert/tokenizer/special_tokens_map.json',
 '/content/drive/MyDrive/ModernEgyBert/tokenizer/tokenizer.json')

In [ ]:
new_tokenizer = AutoTokenizer.from_pretrained(f"{config.base_dir}/tokenizer", use_fast=True)

In [ ]:
def tokenize_function(examples):
    return new_tokenizer(
        examples["text"],
        truncation=True,
        padding="max_length",
        max_length=config.max_length
    )


In [ ]:
column_names = dataset["train"].column_names
train_dataset = dataset["train"].map(
    tokenize_function,
    batched=True,
    remove_columns=column_names
)
test_dataset = dataset["test"].map(
    tokenize_function,
    batched=True,
    remove_columns=column_names
)

print(train_dataset)
print(test_dataset)

Map:   0%|          | 0/1410134 [00:00<?, ? examples/s]

Map:   0%|          | 0/9216 [00:00<?, ? examples/s]

Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 1410134
})
Dataset({
    features: ['input_ids', 'attention_mask'],
    num_rows: 9216
})


In [ ]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=new_tokenizer,
    mlm=True,
    mlm_probability=config.mlm_probability
)

model = AutoModelForMaskedLM.from_pretrained(config.model_name)


In [ ]:
model.resize_token_embeddings(config.new_tokenizer_vocab_size)

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


Embedding(64000, 768, padding_idx=50283)

In [ ]:
# 1) Access the embeddings module
embedding_layer = model.get_input_embeddings()

# 2) Check the shape
# embedding_layer.weight.shape is typically [vocab_size, embedding_dim]
vocab_size, embedding_dim = embedding_layer.weight.shape
print(f"Vocab size: {vocab_size}")
print(f"Embedding dimension: {embedding_dim}")

# 3) (Optional) total number of parameters in embeddings
total_params = embedding_layer.weight.numel()
print(f"Total embedding parameters: {total_params}")

Vocab size: 64000
Embedding dimension: 768
Total embedding parameters: 49152000


In [ ]:
training_args = TrainingArguments(
    output_dir=config.output_dir,
    overwrite_output_dir=config.overwrite_output_dir,
    num_train_epochs=config.num_train_epochs,
    per_device_train_batch_size=config.per_device_train_batch_size,
    per_device_eval_batch_size=config.per_device_eval_batch_size,
    evaluation_strategy=config.evaluation_strategy,
    eval_steps=config.eval_steps,
    logging_steps=config.logging_steps,
    save_steps=config.save_steps,
    save_total_limit=config.save_total_limit,
    learning_rate=config.learning_rate,
    warmup_steps=config.warmup_steps,
    weight_decay=config.weight_decay,
    report_to=config.report_to,
    run_name=config.run_name,
)


/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1576: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator
)

In [19]:
trainer.train() # 60K ++

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


Step,Training Loss,Validation Loss
5000,2.925900,2.954429
10000,2.888500,2.883139
15000,2.798500,2.818218
20000,2.762600,2.778290
25000,2.700600,2.741833
30000,2.643600,2.685061
35000,2.576700,2.658691
40000,2.579200,2.613238
45000,2.539700,2.587728
50000,2.487700,2.567012


Step,Training Loss,Validation Loss
5000,2.925900,2.954429
10000,2.888500,2.883139
15000,2.798500,2.818218
20000,2.762600,2.778290
25000,2.700600,2.741833
30000,2.643600,2.685061
35000,2.576700,2.658691
40000,2.579200,2.613238
45000,2.539700,2.587728
50000,2.487700,2.567012


KeyboardInterrupt: 

In [20]:
from huggingface_hub import login

login(token="your token")


In [21]:
trainer.save_model(config.output_dir)
new_tokenizer.save_pretrained(config.output_dir)

#Push to Hugging Face Hub:
trainer.push_to_hub("Modern-EgyBert-Base")

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/5.43k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/641M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/metga97/egyptian_modernbert_fineweb2/commit/e9662d9b3d2ea1f4ab0ac63c25ff3aa6911ef391', commit_message='Modern-EgyBert-Base', commit_description='', oid='e9662d9b3d2ea1f4ab0ac63c25ff3aa6911ef391', pr_url=None, repo_url=RepoUrl('https://huggingface.co/metga97/egyptian_modernbert_fineweb2', endpoint='https://huggingface.co', repo_type='model', repo_id='metga97/egyptian_modernbert_fineweb2'), pr_revision=None, pr_num=None)

In [24]:
# ===================================
# 10. Quick Mask-Filling Test
# ===================================
from transformers import pipeline

# load from the newly saved folder
fill_mask = pipeline("fill-mask", model="metga97/egyptian_modernbert_fineweb2")

Device set to use cuda:0


In [34]:
demo_text = "  صباح الفل يا [MASK]"
predictions = fill_mask(demo_text)
for p in predictions:
    print(f"{p['sequence']} (score: {p['score']:.4f})")

  صباح الفل يا حبيبي (score: 0.0423)
  صباح الفل يا مصر (score: 0.0379)
  صباح الفل يا مصري (score: 0.0305)
  صباح الفل يا صباح (score: 0.0244)
  صباح الفل يا جميل (score: 0.0230)


In [22]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForMaskedLM

tokenizer = AutoTokenizer.from_pretrained("metga97/egyptian_modernbert_fineweb2")
model = AutoModelForMaskedLM.from_pretrained("metga97/egyptian_modernbert_fineweb2")

tokenizer_config.json:   0%|          | 0.00/1.55k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/6.27M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/694 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/641M [00:00<?, ?B/s]